# normal sheet 1~2 (3부터는 서울시 자료 없음)

In [1]:
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
normal_data_origin = pd.read_excel("../data/normal_data_6.xlsx")
normal_data_origin.head(5)

,개방서비스명,인허가일자,폐업일자,소재지전체주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y)
0,일반음식점,20200506.0,NaN,전라남도 광양시 광양읍 용강리 879-3번지,29억,호프/통닭,255486.864847,163842.721328
1,일반음식점,20200511.0,NaN,전라남도 광양시 광양읍 덕례리 1758-8번지,1℃맥주 덕례점,호프/통닭,251777.700000,163382.050000
2,일반음식점,20200511.0,NaN,전라남도 광양시 중동 1755-10번지,한우명가,식육(숯불구이),263893.686904,159911.195440
3,일반음식점,20200420.0,NaN,전라남도 광양시 광양읍 덕례리 1773-5번지,노다지뷔페,뷔페식,251564.043956,163395.346926
4,일반음식점,20200518.0,NaN,전라남도 광양시 중동 1390-5번지,금자네포차,한식,262811.242376,160540.036356


In [3]:
normal_data = pd.DataFrame({'인허가일자':normal_data_origin['인허가일자'], '폐업일자':normal_data_origin['폐업일자'], '소재지전체주소':normal_data_origin['소재지전체주소'], '업태구분명':normal_data_origin['업태구분명'], 'x':normal_data_origin['좌표정보(X)'], 'y':normal_data_origin['좌표정보(Y)']})
normal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   인허가일자    299979 non-null  float64
 1   폐업일자     175725 non-null  float64
 2   소재지전체주소  298496 non-null  object 
 3   업태구분명    299973 non-null  object 
 4   x        283891 non-null  float64
 5   y        283891 non-null  float64
dtypes: float64(4), object(2)
memory usage: 13.7+ MB


In [4]:
# '인허가일자'와 '소재지전체주소'에 있는 결측치를 제거
normal_data['인허가일자'].dropna()

0         20200506.0
1         20200511.0
2         20200511.0
3         20200420.0
4         20200518.0
             ...    
299995    20090304.0
299996    20090309.0
299997    20090309.0
299998    20090309.0
299999    20020929.0
Name: 인허가일자, Length: 299979, dtype: float64

In [5]:
normal_data['소재지전체주소'].dropna()

0          전라남도 광양시 광양읍 용강리 879-3번지 
1         전라남도 광양시 광양읍 덕례리 1758-8번지 
2             전라남도 광양시 중동 1755-10번지 
3         전라남도 광양시 광양읍 덕례리 1773-5번지 
4              전라남도 광양시 중동 1390-5번지 
                     ...            
299995           충청남도 당진시 수청동 987번지 
299996         충청남도 당진시 읍내동 142-3번지 
299997    충청남도 당진시 석문면 교로리 937-40번지 
299998    충청남도 당진시 송악읍 복운리 1638-7번지 
299999          충청남도 당진시 대덕동 1488번지 
Name: 소재지전체주소, Length: 298496, dtype: object

In [6]:
# 제거한 후에는 index가 꼬이므로 index 재설정
normal_data.reset_index(drop=True, inplace=True)

In [7]:
# 인허가일자 type을 float에서 str로 변환 (19931101.0 > 19931101)
normal_data['인허가일자'] = normal_data['인허가일자'].astype(str).apply(lambda x: x.replace('.0',""))
normal_data['인허가일자']

0         20200506
1         20200511
2         20200511
3         20200420
4         20200518
            ...   
299995    20090304
299996    20090309
299997    20090309
299998    20090309
299999    20020929
Name: 인허가일자, Length: 300000, dtype: object

In [8]:
# 중간중간에 끼어있는 "nan"값을 삭제
normal_data = normal_data.drop(normal_data[normal_data['인허가일자'].map(len)==3].index)
# 삭제 후 인덱스 재정렬
normal_data.reset_index(drop=True, inplace=True)

In [9]:
# 에러가 발생하는지 발생하지 않는지 판별하는 for문
c = normal_data['인허가일자'].values.tolist()
for i in c:
    if (len(i)!=8 or i=='nan'):
        print(c.index(i),i,"에러 발생")

In [ ]:
normal_data.iloc[238503,0] = '19870530'

In [12]:
# 인허가일자 형식을 19931101 > 1993-11-01로 변환
normal_data.iloc[:,0] = normal_data.iloc[:,0].apply(lambda x: x[0:4]+"-"+x[4:6]+"-"+x[6:8])
normal_data.iloc[:,0]

0         2020-05-06
1         2020-05-11
2         2020-05-11
3         2020-04-20
4         2020-05-18
             ...    
299974    2009-03-04
299975    2009-03-09
299976    2009-03-09
299977    2009-03-09
299978    2002-09-29
Name: 인허가일자, Length: 299979, dtype: object

In [19]:
# datetime형식으로 변환
normal_data.iloc[:,0] = pd.to_datetime(normal_data.iloc[:,0])
normal_data.iloc[:,0]

0        2020-05-06
1        2020-05-11
2        2020-05-11
3        2020-04-20
4        2020-05-18
            ...    
299974   2009-03-04
299975   2009-03-09
299976   2009-03-09
299977   2009-03-09
299978   2002-09-29
Name: 인허가일자, Length: 299977, dtype: datetime64[ns]

In [17]:
normal_data[normal_data.iloc[:,0]=="9187-06-08"]

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
98835,9187-06-08,20100111.0,경상북도 상주시 남성동 12-13번지,중국식,303939.392542,324361.420275


In [ ]:
# 앞자리가 1이나 2가 아닌 이상치를 찾는 for문
a = normal_data.iloc[:,0].values.tolist()
a[0][0]
for i in a:
    if (i[0] == "3"):
        print(i, a.index(i),"에러")

In [18]:
normal_data = normal_data.drop(normal_data[normal_data.iloc[:,0]=="9187-06-08"].index)
normal_data

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
0,2020-05-06,NaN,전라남도 광양시 광양읍 용강리 879-3번지,호프/통닭,255486.864847,163842.721328
1,2020-05-11,NaN,전라남도 광양시 광양읍 덕례리 1758-8번지,호프/통닭,251777.700000,163382.050000
2,2020-05-11,NaN,전라남도 광양시 중동 1755-10번지,식육(숯불구이),263893.686904,159911.195440
3,2020-04-20,NaN,전라남도 광양시 광양읍 덕례리 1773-5번지,뷔페식,251564.043956,163395.346926
4,2020-05-18,NaN,전라남도 광양시 중동 1390-5번지,한식,262811.242376,160540.036356
...,...,...,...,...,...,...
299974,2009-03-04,NaN,충청남도 당진시 수청동 987번지,한식,168356.584665,377797.343165
299975,2009-03-09,NaN,충청남도 당진시 읍내동 142-3번지,한식,167004.096833,376369.881780
299976,2009-03-09,NaN,충청남도 당진시 석문면 교로리 937-40번지,한식,155845.726147,394310.728290
299977,2009-03-09,NaN,충청남도 당진시 송악읍 복운리 1638-7번지,한식,180588.965942,382277.917481


In [20]:
# 인허가일자를 index로 만들기(인허가일자가 index가 되어야 날짜별로 slicing이 가능해짐)
normal_data_p = normal_data.set_index(normal_data['인허가일자'], drop=False, append=False, inplace=False)
# inplace=False 할 경우 다른 변수에 저장해야 값이 보존됨

In [21]:
# 2015년부터 2020년 1분기까지의 데이터만 남겨놓기
normal_data_1520 = normal_data_p['2015-01-01':'2020-03-31']
normal_data_1520.to_excel("normal_data_1520_6.xlsx")

In [22]:
normal_data_1520.reset_index(drop=True, inplace=False)

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y
0,2020-02-18,NaN,전라남도 광양시 광양읍 덕례리 1728-1번지,기타,252344.359272,162999.054094
1,2020-02-18,NaN,전라남도 광양시 중동 1883-2번지,한식,263765.088902,158925.821130
2,2020-02-07,NaN,전라남도 광양시 광영동 785-3번지,호프/통닭,265467.783564,162925.396743
3,2020-02-11,NaN,전라남도 광양시 중동 1878-1번지,한식,263784.323610,158984.979359
4,2020-02-25,NaN,전라남도 광양시 광양읍 구산리 846-12번지,한식,252615.402392,164230.233379
...,...,...,...,...,...,...
57450,2020-01-15,NaN,충청남도 당진시 면천면 성상리 934-2번지,한식,170123.685771,368375.025719
57451,2020-03-19,NaN,충청남도 당진시 읍내동 526-2번지,기타,166946.203009,377052.336903
57452,2016-12-23,20200508.0,충청남도 당진시 송악읍 가학리 311-2번지,한식,173553.909824,379339.020787
57453,2016-12-27,20180521.0,충청남도 당진시 읍내동 1545번지,한식,167106.457961,377831.866088


In [ ]:
normal_data_1520.drop(['인허가일자.1'],axis=1,inplace=True)

In [ ]:
normal_data_1520

In [23]:
# 소재지전체주소가 nan값인지 아닌지 판별하는 함수 (nan값이면 해당 값 대신에 true를 넣는다.)
def isnan_not(x):
    for i in x:
        try:
            x[x.index(i)] = math.isnan(i)
        except TypeError:
            x[x.index(i)] = False

In [24]:
alpha = normal_data_1520['소재지전체주소'].values.tolist()
isnan_not(alpha)

In [25]:
normal_data_1520.insert(6,'소재지전체주소_new',alpha)
normal_data_1520

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new
인허가일자,,,,,,,
2020-02-18,2020-02-18,NaN,전라남도 광양시 광양읍 덕례리 1728-1번지,기타,252344.359272,162999.054094,False
2020-02-18,2020-02-18,NaN,전라남도 광양시 중동 1883-2번지,한식,263765.088902,158925.821130,False
2020-02-07,2020-02-07,NaN,전라남도 광양시 광영동 785-3번지,호프/통닭,265467.783564,162925.396743,False
2020-02-11,2020-02-11,NaN,전라남도 광양시 중동 1878-1번지,한식,263784.323610,158984.979359,False
2020-02-25,2020-02-25,NaN,전라남도 광양시 광양읍 구산리 846-12번지,한식,252615.402392,164230.233379,False
...,...,...,...,...,...,...,...
2020-01-15,2020-01-15,NaN,충청남도 당진시 면천면 성상리 934-2번지,한식,170123.685771,368375.025719,False
2020-03-19,2020-03-19,NaN,충청남도 당진시 읍내동 526-2번지,기타,166946.203009,377052.336903,False
2016-12-23,2016-12-23,20200508.0,충청남도 당진시 송악읍 가학리 311-2번지,한식,173553.909824,379339.020787,False


In [26]:
normal_data_1520[normal_data_1520['소재지전체주소_new']==True]

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new
인허가일자,,,,,,,
2017-02-16,2017-02-16,NaN,NaN,호프/통닭,262852.475263,160721.203547,True
2017-02-10,2017-02-10,NaN,NaN,한식,263865.000000,159547.000000,True
2015-12-07,2015-12-07,NaN,NaN,분식,253828.613359,163754.557609,True
2015-11-27,2015-11-27,NaN,NaN,기타,253828.613359,163754.557609,True
2015-04-22,2015-04-22,20180116.0,NaN,까페,198078.486139,202107.370398,True
...,...,...,...,...,...,...,...
2016-07-15,2016-07-15,20170207.0,NaN,분식,168379.859855,378140.468797,True
2015-01-09,2015-01-09,20190117.0,NaN,호프/통닭,179855.131400,376099.727032,True
2016-01-20,2016-01-20,20180102.0,NaN,중국식,NaN,NaN,True


In [27]:
# 소재지전체주소_new가 True인 값(nan값인 경우) 삭제한다.
normal_data_1520 = normal_data_1520[normal_data_1520['소재지전체주소_new']==False]

In [ ]:
normal_data_1520

In [28]:
# 서울특별시가 아니면 True를, 맞으면 False를 넣음
beta = normal_data_1520['소재지전체주소'].values.tolist()
for i in beta:
    if i.split(" ")[0]!="서울특별시":
        beta[beta.index(i)] = True
    else:
        beta[beta.index(i)] = False
print(beta)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [29]:
normal_data_1520.insert(7,'notseoul',beta)

In [ ]:
normal_data_1520

In [30]:
normal_data_1520 = normal_data_1520[normal_data_1520['notseoul']==False]
normal_data_1520

,인허가일자,폐업일자,소재지전체주소,업태구분명,x,y,소재지전체주소_new,notseoul
인허가일자,,,,,,,,


In [ ]:
normal_data_1520.drop(['소재지전체주소_new', 'notseoul'],axis=1,inplace=True)
normal_data_1520

In [ ]:
normal_data_1520.insert(6,'구',normal_data_1520['소재지전체주소'].apply(lambda x: x.split(" ")[1]))

In [ ]:
normal_data_1520.insert(7,'동', normal_data_1520['소재지전체주소'].apply(lambda x: x.split(" ")[2]))
normal_data_1520

In [ ]:
# '가'이나 '로'로 끝나는 값을 찾아내기
omega = normal_data_1520['동'].values
empty = []
for i in omega:
    if (i[-1]=="가" or i[-1]=="로"):
        empty.append(i)

In [ ]:
empty_set = set(empty)
print(empty_set) # unique한 set을 찾기

In [ ]:
dong = normal_data_1520['동'].values.tolist()

In [ ]:
for i in dong:
    if (i in ["종로5가", "종로6가"]):
        dong[dong.index(i)]="종로5.6가동"
    elif (i in ["통의동", "적선동", "체부동", "필운동", "내자동", "사직동", "도렴동", "당주동", "내수동", "세종로", "신문로1가", "신문로2가"]):
        dong[dong.index(i)]="사직동"
    elif (i in ['명륜1가', '명륜2가', '명륜3가', '명륜4가']):
        dong[dong.index(i)]="혜화동"
    elif (i in ['원효로1가', '원효로2가', '문배동', '신계동']):
        dong[dong.index(i)]="원효로1동"
    elif (i in ['신창동', '산천동', '청암동', '원효로3가', '원효로4가']):
        dong[dong.index(i)]="원효로2동"
    elif (i in ['당산동1가', '당산동2가', '당산동3가']):
        dong[dong.index(i)]="당산1동"
    elif (i in ['당산동4가', '당산동5가', '당산동6가', '당산동']):
        dong[dong.index(i)]="당산2동"
    elif (i in ['돈암동', '동소문동6가', '동소문동7가', '동선동1가', '동선동2가', '동선동3가', '동선동4가', '동선동5가']):
        dong[dong.index(i)]="동선동"
    elif (i in ['남창동', '남대문로3가', '남대문로4가', '남대문로5가', '봉래동1가', '봉래동2가', '회현동1가', '회현동2가', '충무로1가', '순화동']):
        dong[dong.index(i)]="회현동"
    elif (i in ['무교동', '다동', '태평로1가', '을지로1가', '을지로2가', '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '남대문로2가', '회현동1가', '회현동2가', '회현동3가', '충무로1가', '충무로2가', '명동1가', '명동2가', '남산동1가', '남산동2가', '남산동3가', '저동1가', '예장동']):
        dong[dong.index(i)]="명동"
    elif (i in ['한강로1가', '한강로2가', '용산동3가', '용산동5가', '한강로3가']):
        dong[dong.index(i)]="한강로동"
    elif (i in ['을지로4가', '을지로5가', '주교동', '방산동', '을지로3가', '입정동', '산림동', '초동', '인현동1가', '저동2가']):
        dong[dong.index(i)]="을지로동"
    elif (i in ['돈암동', '동소문동2가', '동소문동3가', '동소문동5가', '삼선동1가', '삼선동2가', '삼선동3가', '삼선동4가', '삼선동5가']):
        dong[dong.index(i)]="삼선동"
    elif (i in ['세종로', '청진동', '서린동', '수송동', '중학동', '종로1가', '공평동', '관훈동', '견지동', '와룡동', '권농동', '운니동', '익선동', '경운동', '관철동', '인사동', '낙원동', '종로2가', '훈정동', '묘동', '봉익동', '돈의동', '장사동', '관수동', '종로3가', '인의동', '예지동', '원남동', '종로4가']):
        dong[dong.index(i)]="종로1.2.3.4가동"
    elif (i in ['보문동4가', '보문동5가', '보문동6가', '보문동7가', '보문동1가', '보문동2가', '보문동3가']):
        dong[dong.index(i)]="보문동"
    elif (i in ['금호동4가']):
        dong[dong.index(i)]="금호4가동"
    elif (i in ['금호동2가', '금호동3가']):
        dong[dong.index(i)]="금호2.3가동"
    elif (i in ['금호동1가']):
        dong[dong.index(i)]="금호1가동"
    elif (i in ['영등포동1가', '영등포동2가', '영등포동3가', '영등포동4가', '영등포동5가', '영등포동6가', '영등포동7가', '영등포동8가']):
        dong[dong.index(i)]="영등포동"
    elif (i in ['영등포동', '신길동']):
        dong[dong.index(i)]="영등포본동"
    elif (i in ['충무로4가', '충무로5가', '묵정동', '필동1가', '필동2가', '필동3가', '남학동', '주자동', '예장동', '장충동2가', '충무로3가']):
        dong[dong.index(i)]="필동"
    elif (i in ['소공동', '북창동', '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '봉래동1가', '서소문동', '정동', '순화동', '의주로1가', '충정로1가']):
        dong[dong.index(i)]="소공동"
    elif (i in ['충무로4가', '충무로5가', '인현동2가', '예관동', '광희동1가', '광희동2가', '쌍림동', '을지로6가', '을지로7가', '오장동']):
        dong[dong.index(i)]="광희동"
    elif (i in ['안암동1가', '안암동2가', '안암동3가', '안암동4가', '안암동5가']):
        dong[dong.index(i)]="안암동"
    elif (i in ['서계동', '청파동1가', '청파동2가', '청파동3가']):
        dong[dong.index(i)]="청파동"
    elif (i in ['양평동1가', '양평동2가', '양평동3가']):
        dong[dong.index(i)]="양평1동"
    elif (i in ['양평동3가', '양평동4가', '양평동5가', '양평동6가', '양화동', '양평동']):
        dong[dong.index(i)]="양평2동"
    elif (i in ['용산동2가', '용산동4가']):
        dong[dong.index(i)]="용산2가동"
    elif (i in ['성수동1가']):
        dong[dong.index(i)]="성수1가1동"
    elif (i in ['성수동2가']):
        dong[dong.index(i)]="성수2가1동"
    elif (i in ['성북동', '성북동1가', '동소문동1가', '동소문동4가']):
        dong[dong.index(i)]="성북동"
    elif (i in ['문래동1가', '문래동2가', '문래동3가', '문래동4가', '문래동5가', '문래동6가']):
        dong[dong.index(i)]="문래동"
    elif (i in ['원효로1가', '원효로2가', '문배동', '신계동']):
        dong[dong.index(i)]="원효로1동"
    elif (i in ['신창동', '산천동', '청암동', '원효로3가', '원효로4가']):
        dong[dong.index(i)]="원효로2동"
    elif (i in ['중림동', '의주로2가', '만리동1가', '만리동2가']):
        dong[dong.index(i)]="중림동"
    elif (i in ['충정로2가', '충정로3가', '합동', '미근동', '북아현동']):
        dong[dong.index(i)]="충현동"
    elif (i in ['묵정동', '장충동1가', '장충동2가']):
        dong[dong.index(i)]="장충동"
    elif (i in ['동빙고동', '서빙고동', '주성동', '용산동6가']):
        dong[dong.index(i)]="서빙고동"
    elif (i in ['갈월동', '남영동', '용산동1가', '동자동']):
        dong[dong.index(i)]="남영동"
print(dong)

In [ ]:
# 돌리고 나서 빼먹은 부분이 있는지 찾기
for i in dong:
    if (i[-1]=="가" or i[-1]=="로"):
        print(i)

In [ ]:
normal_data_1520.insert(8,'동_new',dong)
normal_data_1520

In [ ]:
normal_data_1520.iloc[:,1]

In [ ]:
normal_data_1520.iloc[:,1] = normal_data_1520.iloc[:,1].astype(str).apply(lambda x: x.replace('.0',""))
normal_data_1520

In [ ]:
closedstore = normal_data_1520['폐업일자'].values.tolist()
for i in closedstore:
    if i=="nan":
        continue
    else:
        closedstore[closedstore.index(i)] = i[0:4]+"-"+i[4:6]+"-"+i[6:8]
        
print(closedstore)

In [ ]:
normal_data_1520.iloc[:,1] = closedstore

In [ ]:
normal_data_1520

In [ ]:
normal_data_openrate = pd.DataFrame({'인허가일자':normal_data_1520['인허가일자'], '구':normal_data_1520['구'], '동':normal_data_1520['동_new'], 'x':normal_data_1520['x'],'y':normal_data_1520['y']})

In [ ]:
normal_data_openrate

In [ ]:
def changeyear_opened():
    for i in [2015, 2016, 2017, 2018, 2019, 2020]:
        cnt_1 = []
        openday = normal_data_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,1,1) and pd.to_datetime(j) <= datetime.datetime(i,3,31)):
                cnt_1.append(1)
            else:
                cnt_1.append(0)
        normal_data_openrate[str(i)+"_1"] = cnt_1

        cnt_2 = []
        openday = normal_data_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,4,1) and pd.to_datetime(j) <= datetime.datetime(i,6,30)):
                cnt_2.append(1)
            else:
                cnt_2.append(0)
        normal_data_openrate[str(i)+"_2"] = cnt_2

        cnt_3 = []
        openday = normal_data_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,7,1) and pd.to_datetime(j) <= datetime.datetime(i,9,30)):
                cnt_3.append(1)
            else:
                cnt_3.append(0)
        normal_data_openrate[str(i)+"_3"] = cnt_3

        cnt_4 = []
        openday = normal_data_openrate['인허가일자'].values.tolist()
        for j in openday:
            if (pd.to_datetime(j) >= datetime.datetime(i,10,1) and pd.to_datetime(j) <= datetime.datetime(i,12,31)):
                cnt_4.append(1)
            else:
                cnt_4.append(0)
        normal_data_openrate[str(i)+"_4"] = cnt_4

In [ ]:
changeyear_opened()

In [ ]:
normal_data_openrate

In [ ]:
normal_data_openrate.reset_index(drop=True, inplace=True)

In [ ]:
normal_data_openrate.to_excel("normal_data_2_openrate.xlsx")

In [ ]:
normal_data_closedrate = pd.DataFrame({'폐업일자':normal_data_1520['폐업일자'], '구':normal_data_1520['구'], '동':normal_data_1520['동_new'], 'x':normal_data_1520['x'],'y':normal_data_1520['y']})
normal_data_closedrate

In [ ]:
normal_data_closedrate.iloc[:,0]

In [ ]:
def changeyear_closed():
    for i in [2015, 2016, 2017, 2018, 2019, 2020]:
        cnt_1 = []
        closedday = normal_data_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,1,1) and pd.to_datetime(j) <= datetime.datetime(i,3,31)):
                cnt_1.append(1)
            elif (str(j) == "nan"):
                cnt_1.append(0)
            else:
                cnt_1.append(0)
        normal_data_closedrate[str(i)+"_1"] = cnt_1

        cnt_2 = []
        closedday = normal_data_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,4,1) and pd.to_datetime(j) <= datetime.datetime(i,6,30)):
                cnt_2.append(1)
            elif (str(j) == "nan"):
                cnt_2.append(0)
            else:
                cnt_2.append(0)
        normal_data_closedrate[str(i)+"_2"] = cnt_2

        cnt_3 = []
        closedday = normal_data_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,7,1) and pd.to_datetime(j) <= datetime.datetime(i,9,30)):
                cnt_3.append(1)
            elif (str(j) == "nan"):
                cnt_3.append(0)
            else:
                cnt_3.append(0)
        normal_data_closedrate[str(i)+"_3"] = cnt_3

        cnt_4 = []
        closedday = normal_data_closedrate['폐업일자'].values.tolist()
        for j in closedday:
            if (pd.to_datetime(j) >= datetime.datetime(i,10,1) and pd.to_datetime(j) <= datetime.datetime(i,12,31)):
                cnt_4.append(1)
            elif (str(j) == "nan"):
                cnt_4.append(0)
            else:
                cnt_4.append(0)
        normal_data_closedrate[str(i)+"_4"] = cnt_4

In [ ]:
changeyear_closed()

In [ ]:
normal_data_closedrate

In [ ]:
normal_data_closedrate.drop(['2020_2','2020_3','2020_4'], axis=1, inplace=True)
normal_data_closedrate
normal_data_closedrate.to_excel("normal_data_closedrate.xlsx")

In [ ]:
normal_data_closedrate.to_excel("normal_data_2_closedrate.xlsx")